In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from Map import Map
from Tabu import Tabu
from TwoStage import TwoStage
from L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl

**Modelling Parameters**

In [3]:
rides = 5
bus = 2
scenarios = 25
MIPGap = 0.001
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [4]:
mappy = Map(rides)

In [5]:
def Mod3():
    t3 = time.time()
    lshaped = mp(
        mappy,
        bus=bus,
        scenarios=scenarios,
#         probability=probability
    )
    lshaped.initialize()
    lshaped.model.params.MIPGap = MIPGap
    lshaped.optimize()
    t3 = time.time() - t3
    return lshaped, t3

**Plotting Functions**

In [6]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


In [7]:
lshaped, t3 = Mod3()

Using license file C:\Users\Lavkesh\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 2
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter Heuristics unchanged
   Value: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter MIPGap to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
-14.618781801960722 -6.837659379568563
52.05717606043865 44.27605363804649
-6.404484037622751 -0.3400408778407229
50.34049679782952 44.27605363804749
-2.8909579027913423 2.1414282587923488
49.308439799630676 44.27605363804699
1.5961905218064203 5.0490992744082925
47.72896239064887 44.276053638046996
8.810519846238094 10.415427262712337
46.30511022224366 44.70020280576942
6.30538052

AttributeError: Unable to retrieve attribute 'ObjVal'

H    0     0                     296.7100271   46.60814  84.3%     -    5s
     0     0   46.61160    0   56  296.71003   46.61160  84.3%     -    5s
     0     0   46.61200    0   58  296.71003   46.61200  84.3%     -    5s


Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in OptCutFun
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in <genexpr>
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "model.pxi", line 343, in gurobipy.Model.__getattr__
  File "model.pxi", line 1772, in gurobipy.Model.getAttr
  File "attrutil.pxi", line 100, in gurobipy.__getattr
AttributeError: Unable to retrieve attribute 'ObjVal'


     0     0   46.62431    0   51  296.71003   46.62431  84.3%     -    5s
     0     0   46.62657    0   49  296.71003   46.62657  84.3%     -    5s
     0     0   46.62865    0   50  296.71003   46.62865  84.3%     -    5s
     0     0   46.62883    0   50  296.71003   46.62883  84.3%     -    5s
     0     0   46.63424    0   51  296.71003   46.63424  84.3%     -    5s
     0     0   46.63614    0   53  296.71003   46.63614  84.3%     -    5s
     0     0   46.63644    0   53  296.71003   46.63644  84.3%     -    5s
     0     0   46.64259    0   47  296.71003   46.64259  84.3%     -    6s
     0     0   46.64259    0   47  296.71003   46.64259  84.3%     -    6s
     0     2   46.64259    0   47  296.71003   46.64259  84.3%     -    6s


...

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in OptCutFun
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in <genexpr>
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "model.pxi", line 343, in gurobipy.Model.__getattr__
  File "model.pxi", line 1772, in gurobipy.Model.getAttr
  File "attrutil.pxi", line 100, in gurobipy.__getattr
AttributeError: Unable to retrieve attribute 'ObjVal'


AttributeError: Unable to retrieve attribute 'ObjVal'

*  327    26              23      66.4157479   47.61503  28.3%  20.7   16s

Cutting planes:
  Gomory: 33
  Cover: 1
  Implied bound: 12
  Projected implied bound: 2
  MIR: 16
  Flow cover: 207
  Zero half: 97
  RLT: 27
  Relax-and-lift: 8

Explored 449 nodes (10808 simplex iterations) in 16.27 seconds
Thread count was 8 (of 8 available processors)

Solution count 6: 66.4157 68.6217 70.6296 ... 296.71

Optimal solution found (tolerance 1.00e-03)
Best objective 6.641574789397e+01, best bound 6.641574789397e+01, gap 0.0000%

User-callback calls 2673, time in user-callback 11.33 sec


Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in OptCutFun
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "C:\Users\Lavkesh\github\DARP\L_shaped.py", line 98, in <genexpr>
    z_sub = sum(self.scenarioprob[s] * self.submodel[s].model.ObjVal for s in range(self.scenarios))
  File "model.pxi", line 343, in gurobipy.Model.__getattr__
  File "model.pxi", line 1772, in gurobipy.Model.getAttr
  File "attrutil.pxi", line 100, in gurobipy.__getattr
AttributeError: Unable to retrieve attribute 'ObjVal'


AttributeError: Unable to retrieve attribute 'ObjVal'

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

In [ ]:
plot_trend(lshaped.upperbounds[1:],lshaped.lowerbounds[1:],0)

In [ ]:
lshaped.printsol(lshaped.submodel)

In [ ]:
(sum(lshaped.variables.h[i,j].X for i,j in lshaped.variables.h.keys()),
sum(lshaped.variables.p_l[j].X for j in lshaped.variables.p_l.keys()))

In [ ]:
lshaped.submodel[3].sim.alpha